# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
sys.path.insert(1, '..')
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,poli,20,TZ,1590356558,88,-3.33,36.80,68.00,11.41
1,1,lata,100,PT,1590356558,84,40.16,-8.33,64.99,1.99
2,2,severo-kurilsk,100,RU,1590356558,90,50.68,156.12,39.94,7.85
3,3,dikson,82,RU,1590356558,97,73.51,80.55,29.41,8.88
4,4,yellowknife,20,CA,1590356559,45,62.46,-114.35,46.40,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [10]:

 # Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
#Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
128,128,namibe,0,AO,1590356595,64,-15.20,12.15,74.35,6.53
169,169,sao filipe,0,CV,1590356608,72,14.90,-24.50,73.99,8.30
204,204,hazorasp,0,UZ,1590356618,36,41.32,61.07,75.20,6.93
218,218,kaka,0,TM,1590356623,41,37.35,59.61,77.59,8.52
294,294,caravelas,0,BR,1590356646,93,-17.71,-39.25,74.77,3.11
361,361,santiago,0,CL,1590356387,17,-33.46,-70.65,78.80,3.36
411,411,itarema,0,BR,1590356681,88,-2.92,-39.92,78.51,7.43
428,428,deolali,0,IN,1590356686,59,19.95,73.83,77.31,7.83
448,448,benguela,0,AO,1590356691,82,-12.58,13.41,73.54,1.90
458,458,oussouye,0,SN,1590356693,83,12.48,-16.55,77.00,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
128,namibe,AO,-15.20,12.15,
169,sao filipe,CV,14.90,-24.50,
204,hazorasp,UZ,41.32,61.07,
218,kaka,TM,37.35,59.61,
294,caravelas,BR,-17.71,-39.25,
361,santiago,CL,-33.46,-70.65,
411,itarema,BR,-2.92,-39.92,
428,deolali,IN,19.95,73.83,
448,benguela,AO,-12.58,13.41,
458,oussouye,SN,12.48,-16.55,


In [16]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = str(lat)+","+str(lng)
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
128,namibe,AO,-15.20,12.15,Chik-Chik Namibe
169,sao filipe,CV,14.90,-24.50,Tortuga B&B
204,hazorasp,UZ,41.32,61.07,Kemping wifi hotel
218,kaka,TM,37.35,59.61,
294,caravelas,BR,-17.71,-39.25,Pousada dos Navegantes
361,santiago,CL,-33.46,-70.65,Hotel Panamericano
411,itarema,BR,-2.92,-39.92,Windvillage
428,deolali,IN,19.95,73.83,Quality Inn Regency
448,benguela,AO,-12.58,13.41,Hotel Praia Morena
458,oussouye,SN,12.48,-16.55,Auberge du Routard


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [18]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)



# Display Map
fig

NameError: name 'hotel_info' is not defined